In [ ]:
import pandas as pd
import numpy as np
from functools import reduce
from sklearn import preprocessing

In [ ]:
df = pd.read_csv("sapatos.csv", header=0)[['id', 'brand', 'categories', 'primaryCategories', 'colors',
       'prices.amountMax', 'prices.amountMin', 'prices.color',
       'prices.condition', 'prices.isSale']]

In [ ]:
df["prices.condition"] = [condition.upper() for condition in df["prices.condition"].fillna("USED")]

In [ ]:
df = df.drop(["primaryCategories"], axis=1)

In [ ]:
df["colors"] = df["colors"].fillna("N")
df["colors"] = [tuple(names.replace("/", " ").split(" ")[0].strip() for names in set(colors.split(","))) for colors in df["colors"].fillna("")]
df["categories"] = [tuple(category.strip() for category in set(categories.split(","))) for categories in df["categories"].fillna("")]

In [ ]:
categories = set(reduce(lambda a,b: a+b, df["categories"].unique()))
colors = set(reduce(lambda a,b: a+b, df["colors"].unique()))

In [ ]:
colors

In [ ]:
categories

In [ ]:
df['id'].nunique()

In [ ]:
df["id"].count()

In [ ]:
df["prices.amountMax"] = [float(str(price).replace(",", ".")) for price in df["prices.amountMax"]]
df["prices.amountMin"] = [float(str(price).replace(",", ".")) for price in df["prices.amountMin"]]
df["price"] = (df["prices.amountMax"] + df["prices.amountMin"])/2
df = df.drop(["prices.amountMax", "prices.amountMin", "prices.color"], axis=1)

In [ ]:
df = df.rename(columns={"prices.condition": "condition", "prices.isSale": "isSale"})

In [ ]:
df=df.drop_duplicates()

In [ ]:
#Você deve usar o algoritmo de sua preferência para agrupar os sapatos de acordo com
#condition, categories e colors disponíveis gerando 4 clusters.

# Transforma a coluna df['condition'] de strings para 0 ou 1
le = preprocessing.LabelEncoder()
condition_list = df['condition']
le.fit(condition_list)
le.classes_
df['condition']=le.transform(condition_list)
# Tira a média dos preços para cada grupo de id
df_means= df.drop(['brand','categories','colors','isSale'],axis=1)
df_means=df_means.groupby(df['id']).mean()
# Seleciona uma linha de id para as outras categorias, pois os dados se repetem para cada id.
df_others = df.drop(['condition','isSale','price'],axis=1)
df_others = df_others.drop_duplicates('id')

In [ ]:
df_dummies=df_others.drop(['id','brand'],axis=1)
df_dummies=pd.get_dummies(df_dummies)
df_dummies=pd.concat([df_others['id'],df_dummies],axis=1)

In [ ]:
df=pd.merge(df_dummies,df_means, on='id', how='left')

In [ ]:
with open('data_cleaned.json', 'w', encoding='utf-8') as file:
    df.to_json(file, force_ascii=False, orient='records', lines=True)